In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('RAM: {:.1f} GB'.format(ram_gb))

RAM: 4.2 GB


In [17]:
import multiprocessing

print(multiprocessing.cpu_count())

4


In [19]:
import os
import glob
import pickle
import datetime

import netCDF4
import json

import numpy as np
import pandas as pd

In [20]:
# input
year_input          = '2024'
path_input_folder   = 'D:\\CYGNSS_L1_V3.2_3.2-20250317_135601\\'

# Coordinate VN - WGS84-Zone0
lat_start, lat_end = 0, 24
lon_start, lon_end = 102, 110 

# ouput
path_output_folder  = 'D:\\raw_vn\\20241\\'

In [23]:
def Present(print = False):
    # get timezone HN (UTC +7)
    tz_hanoi = datetime.timezone(datetime.timedelta(hours=7))
    # get present time
    now     = datetime.datetime.now(tz_hanoi)
    # convert to str ...
    dt_str  = now.strftime("%Y/%m/%d %H:%M")
    dt_str  = dt_str.replace('/', '')
    dt_str  = dt_str.replace(':', 'h')
    dt_str  = dt_str.replace(' ', '_')
    # print???
    if print == True:
        print (dt_str)
    # return
    return dt_str
Present()


'20250319_20h32'

In [25]:
def FilterByCoordinates (lat_, lon_):
    if (lat_ >= lat_start and lat_ <= lat_end) and (lon_ >= lon_start and lon_ <= lon_end):
        return True
    else:
        return False

print(FilterByCoordinates(19,106))
print(FilterByCoordinates(16,107))

#-------------------------------------------------------------------------------
def FilterPerData (data_):
    lats_ = data_.variables['sp_lat'][:]
    lons_ = data_.variables['sp_lon'][:]

    index_valid_ = []
    for i in range (0, len(lats_)):
        for j in range (0, 4):
            lt_ = lats_[i][j]
            ln_ = lons_[i][j]
            if (FilterByCoordinates(lt_, ln_) == True):
                index_valid_.append(i)
                break
            else: continue
    # data_.close()
    return index_valid_
#-------------------------------------------------------------------------------
def getDictAttr (netCDF4Dataset):
    metadata = {}
    for attrname in netCDF4Dataset.ncattrs():
        metadata[attrname] = netCDF4Dataset.getncattr(attrname)
    return metadata

True
True


In [34]:
path_input_files = glob.glob(os.path.join(path_input_folder, "*.nc"))

check_num = 0

for path in path_input_files:
    input_filename_p    = path.split('\\')[-1]
    output_filename_p   = input_filename_p.replace('.nc', '.vn.nc')

    # print
    print('---\n', check_num, input_filename_p)

    with netCDF4.Dataset(path, moder = 'r') as netCDF4_input_p:
        #-----------------------------------------------------------------------
        # Get index valid to my AOI - Area Of Interest
        index_valid_p   = FilterPerData(netCDF4_input_p)

        #-----------------------------------------------------------------------
        # get dimensions of dataset
        dimensions_p    = {dimname: len(netCDF4_input_p.dimensions[dimname]) for dimname in netCDF4_input_p.dimensions.keys()}

        # create dict dimensions valid
        dimensions_p['sample']  = len(index_valid_p)
        dict_dimensions_valid_p = dimensions_p
        list_dimensions_keys_p  = list(dict_dimensions_valid_p.keys())

        #-----------------------------------------------------------------------
        # Get and create dict global variables of CDF4 dataset
        dict_global_variables_p                  = getDictAttr(netCDF4_input_p)
        dict_global_variables_p['lat_filter']    = [lat_start, lat_end]
        dict_global_variables_p['lon_filter']    = [lon_start, lon_end]


        #-----------------------------------------------------------------------
        # Get important variables from CDF4 dataset
        input_variables_p       = netCDF4_input_p.variables
        input_variables_keys_p  = list(input_variables_p.keys())


        # filter important variables by index_valid_p
        dict_important_variables_valid     = {}
        for vkey in input_variables_keys_p: # duyệt theo variables keys của netcdf4 data
            # lấy data từ netcdf dataset
            variables_keys_p_vkey = input_variables_p[vkey]
            try:
                # nếu dimension hợp lý so vs index valid thì thêm vào variables mới
                dict_important_variables_valid[vkey]    = [
                                                            variables_keys_p_vkey.dtype, # cái này là type của variables để tạo variables mới
                                                            variables_keys_p_vkey[:][index_valid_p] # cái này là dữ liệu được trích xuất bằng index_valid
                                                        ]
            except:
                # nếu không thì chuyển cái đó thành global variables - metadata
                dict_global_variables_p[vkey]           = variables_keys_p_vkey[:]

        #-----------------------------------------------------------------------
        # export dataset to new netCDF4 file
        with netCDF4.Dataset(path_output_folder + output_filename_p, "w", format="NETCDF4") as output_file_p:
            #------------------------
            # create global attribute
            output_file_p.setncatts(dict_global_variables_p)

            #------------------------
            # create dimension
            for dim_name, dim_option in dict_dimensions_valid_p.items():
                output_file_p.createDimension(
                                                dim_name,
                                                dim_option
                                            )
            # display(output_file_p)

            #------------------------
            # create important variables
            for var_name, (var_dtype, var_data) in dict_important_variables_valid.items():
                # Lấy thứ tự tên dimensions của biến gốc (nếu có)
                orig_dims = input_variables_p[var_name].dimensions  # Đây là tuple chứa tên các dimension của biến gốc
                dims_to_use = []
                # Với mỗi chiều của var_data, xác định tên dimension sẽ dùng
                for i in range(len(var_data.shape)):
                    if i < len(orig_dims):
                        # Sử dụng tên dimension gốc nếu có
                        dim_name = orig_dims[i]
                    else:
                        # Nếu số chiều của dữ liệu lớn hơn số tên dimensions gốc, tạo tên mới theo thứ tự
                        dim_name = "dim{}".format(i)
                    dims_to_use.append(dim_name)
 
                    # Kiểm tra xem dimension đã tồn tại trong file xuất chưa
                    if dim_name not in output_file_p.dimensions:
                        # Nếu chưa, tạo dimension với kích thước theo var_data.shape[i]
                        output_file_p.createDimension(dim_name, var_data.shape[i])
                    else:
                        # Nếu đã tồn tại nhưng kích thước khác, tạo dimension mới với tên thay đổi
                        if output_file_p.dimensions[dim_name].size != var_data.shape[i]:
                            new_dim_name = dim_name + "_new"
                            dims_to_use[-1] = new_dim_name
                            if new_dim_name not in output_file_p.dimensions:
                                output_file_p.createDimension(new_dim_name, var_data.shape[i])
 
                # Tạo biến mới với tên, kiểu dữ liệu và dimensions đã xác định
                new_var = output_file_p.createVariable(var_name, var_dtype, dimensions=dims_to_use)
 
                # Lấy các thuộc tính của biến gốc và set cho biến mới
                dict_attr_var_i = getDictAttr(input_variables_p[var_name])
                new_var.setncatts(dict_attr_var_i)
 
                # Gán dữ liệu cho biến mới
                new_var[:] = var_data



    check_num += 1


---
 0 cyg01.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.nc
---
 1 cyg01.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.nc
---
 2 cyg02.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.nc
---
 3 cyg02.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.nc
---
 4 cyg03.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.nc
---
 5 cyg03.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.nc
---
 6 cyg04.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.nc
---
 7 cyg04.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.nc
---
 8 cyg05.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.nc
---
 9 cyg05.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.nc
---
 10 cyg07.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.nc
---
 11 cyg07.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.nc
---
 12 cyg08.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33

In [42]:
import arrow
import glob
import netCDF4
import pandas as pd
import numpy as np
check_num = 0
# Thư mục chứa file NetCDF4
folder_path = r"D:\raw_vn\20241"

# Lấy danh sách tất cả các file .nc trong thư mục
nc_files = glob.glob(os.path.join(folder_path, "*.nc"))

pi = 3.14
lamda = 0.19
# Thư mục lưu file CSV
output_folder = r"D:\HThiem\cygnss_to_csv"
os.makedirs(output_folder, exist_ok=True)

# Lặp qua từng file và xuất dữ liệu
for file in nc_files:
    with netCDF4.Dataset(file, mode="r") as nc:

        # Lấy danh sách tất cả biến trong file
        #variables = list(nc.variables.keys())

        # Đọc dữ liệu từ tất cả biến vào DataFrame
        #data_dict = {var: nc.variables[var][:].flatten() for var in variables}

        # Xác định kích thước nhỏ nhất để đồng bộ dữ liệu
        #min_length = min(len(v) for v in data_dict.values())
        #for key in data_dict:
        #    data_dict[key] = data_dict[key][:min_length]

        # Tạo DataFrame từ toàn bộ dữ liệu
        #df = pd.DataFrame(data_dict)
        sp_lat = nc.variables['sp_lat'][:]
        sp_lon = nc.variables['sp_lon'][:]
        ddm_snr = nc.variables['ddm_snr'][:]
        ddm_nbrcs = nc.variables['ddm_nbrcs'][:]
        gps_eirp = nc.variables['gps_eirp'][:]
        tx_to_sp_range = nc.variables['tx_to_sp_range'][:]
        rx_to_sp_range = nc.variables['rx_to_sp_range'][:]
        sp_inc_angle = nc.variables['sp_inc_angle'][:]
        sp_rx_gain = nc.variables['sp_rx_gain'][:]
        quality_flags = nc.variables['quality_flags'][:]
        quality_flags_2 = nc.variables['quality_flags_2'] [:]
        gps_tx_power_db_w = nc.variables['gps_tx_power_db_w'][:]
        gps_ant_gain_db_i = nc.variables['gps_ant_gain_db_i'][:]	
        ddm_noise_floor = nc.variables['ddm_noise_floor'][:]
        ddm_nbrcs_peak = nc.variables['ddm_nbrcs_peak'][:]
        
        df = pd.DataFrame([])
        df['sp_lat'] = sp_lat.flatten()
        df['sp_lon'] = sp_lon.flatten()
        df['ddm_noise_floor'] = ddm_noise_floor.flatten()
        df['ddm_snr'] = ddm_snr.flatten()
        df['gps_tx_power_db_w'] = gps_tx_power_db_w.flatten()
        df['gps_ant_gain_db_i'] = gps_ant_gain_db_i.flatten()
        df['ddm_nbrcs'] = ddm_nbrcs.flatten()
        df['gps_eirp'] = gps_eirp.flatten()
        df['tx_to_sp_range'] = tx_to_sp_range.flatten()
        df['rx_to_sp_range'] = rx_to_sp_range.flatten()
        df['sp_inc_angle'] = sp_inc_angle.flatten()
        df['sp_rx_gain'] = sp_rx_gain.flatten()
        df['quality_flags'] = quality_flags.flatten()
        df['quality_flags_2'] = quality_flags_2.flatten()
        df['ddm_nbrcs_peak'] = ddm_nbrcs_peak.flatten()
        
        #df['SR']=10*np.log10(df['ddm_snr']) - 10*np.log10(df['gps_eirp']) - 10*np.log10(sp_rx_gain) + 20*np.log10(df['tx_to_sp_range']+df['rx_to_sp_range']) + 20*np.log10(4*pi) - 20*np.log10(lamda) 
        #df['SR']= df['SR'] - 10*np.log10(np.cos(np.radian(df['sp_inc_angle'])))
        df = df.replace(-9999, float('nan'))
        # Xuất ra file CSV
        csv_filename = os.path.join(output_folder, os.path.basename(file).replace(".nc", ".csv"))
        df.to_csv(csv_filename, index=False)
        check_num += 1;
        print('---\n', check_num, csv_filename)
 

---
 1 D:\HThiem\cygnss_to_csv\cyg01.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.vn.csv
---
 2 D:\HThiem\cygnss_to_csv\cyg01.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.vn.csv
---
 3 D:\HThiem\cygnss_to_csv\cyg02.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.vn.csv
---
 4 D:\HThiem\cygnss_to_csv\cyg02.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.vn.csv
---
 5 D:\HThiem\cygnss_to_csv\cyg03.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.vn.csv
---
 6 D:\HThiem\cygnss_to_csv\cyg03.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.vn.csv
---
 7 D:\HThiem\cygnss_to_csv\cyg04.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.vn.csv
---
 8 D:\HThiem\cygnss_to_csv\cyg04.ddmi.s20240912-000000-e20240912-235959.l1.power-brcs.a32.d33.vn.csv
---
 9 D:\HThiem\cygnss_to_csv\cyg05.ddmi.s20240911-000000-e20240911-235959.l1.power-brcs.a32.d33.vn.csv
---
 10 D:\HThiem\cygnss_to_csv\cyg05.ddmi.s20240912-00

In [44]:
import os
import pandas as pd
import re
from glob import glob
check_num = 0
csv_folder = "D:\\HThiem\\cygnss_to_csv\\"

csv_files = glob(os.path.join(csv_folder, "*.csv"))

file_groups = {}

for file in csv_files:
    filename = os.path.basename(file)

    match = re.search(r"(\d{8})", filename)
    if match:
        date = match.group(1)

        if date not in file_groups:
            file_groups[date] = []
        file_groups[date].append(file)

for date, files in file_groups.items():
    data_frames = []
    for file in files:
        df_n = pd.read_csv(file)
        df = df_n.dropna(subset=['ddm_snr', 'gps_tx_power_db_w', 'gps_ant_gain_db_i', 'sp_rx_gain', 'tx_to_sp_range', 'rx_to_sp_range', 'sp_inc_angle'])
        df.loc[:, 'SR']=df['ddm_snr'] - 10*np.log10(df['gps_tx_power_db_w']) - 10*np.log10(df['gps_ant_gain_db_i']) - 10*np.log10(df['sp_rx_gain']) + 20*np.log10(df['tx_to_sp_range']+df['rx_to_sp_range']) + 20*np.log10(4*pi) - 20*np.log10(lamda) 
        df.loc[:, 'SR']= df['SR'] - 10*np.log10(np.cos(np.radians(df['sp_inc_angle'])))
        data_frames.append(df)
                
    merged_df = pd.concat(data_frames, ignore_index=True)
    merged_df = merged_df.dropna(subset=['SR'])
    output_file = os.path.join(csv_folder, f"{date}.csv")
    merged_df.to_csv(output_file, index=False)
    check_num += 1
    print(output_file)


C:\Users\DELL\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9244\2472069065.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'SR']=df['ddm_snr'] - 10*np.log10(df['gps_tx_power_db_w']) - 10*np.log10(df['gps_ant_gain_db_i']) - 10*np.log10(df['sp_rx_gain']) + 20*np.log10(df['tx_to_sp_range']+df['rx_to_sp_range']) + 20*np.log10(4*pi) - 20*np.log10(lamda)
C:\Users\DELL\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9244\247206

D:\HThiem\cygnss_to_csv\20240911.csv


C:\Users\DELL\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9244\2472069065.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'SR']=df['ddm_snr'] - 10*np.log10(df['gps_tx_power_db_w']) - 10*np.log10(df['gps_ant_gain_db_i']) - 10*np.log10(df['sp_rx_gain']) + 20*np.log10(df['tx_to_sp_range']+df['rx_to_sp_range']) + 20*np.log10(4*pi) - 20*np.log10(lamda)
C:\Users\DELL\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9244\247206

D:\HThiem\cygnss_to_csv\20240912.csv


In [ ]:
import numpy as np
import os
import pandas as pd
from glob import glob

# Định nghĩa các hằng số
lamda = 0.19
pi = 3.14

# Thư mục chứa các file CSV đầu vào
input_folder = r"D:\HThiem\cygnss_to_csv"
# Thư mục chứa các file CSV đầu ra
output_folder = r"D:\output_csv_sr"

# Tạo thư mục đầu ra nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Lặp qua từng file CSV trong thư mục đầu vào
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):  # Chỉ xử lý file CSV
        file_path = os.path.join(input_folder, filename)

        # Đọc dữ liệu CSV
        df = pd.read_csv(file_path)

        # Kiểm tra các cột cần thiết có trong dữ liệu không
        required_columns = ['ddm_snr', 'gps_eirp', 'sp_rx_gain', 
                            'tx_to_sp_range', 'rx_to_sp_range', 'sp_inc_angle']
        if all(col in df.columns for col in required_columns):
            # Tính SR theo công thức
            df['SR'] = (10 * np.log10(df['ddm_snr']) 
                        - 10 * np.log10(df['gps_eirp']) 
                        - 10 * np.log10(df['sp_rx_gain'])  
                        + 20 * np.log10(df['tx_to_sp_range'] + df['rx_to_sp_range'])  
                        + 20 * np.log10(4 * pi) 
                        - 20 * np.log10(lamda))

            # Thêm hiệu chỉnh góc tới
            df['SR'] = df['SR'] - 10 * np.log10(np.cos(np.radians(df['sp_inc_angle'])))

            # Định nghĩa đường dẫn file CSV đầu ra
            output_path = os.path.join(output_folder, f"sr_{filename}")

            # Lưu kết quả ra file CSV mới
            df.to_csv(output_path, index=False)
            print(f"Đã xử lý và lưu: {output_path}")
        else:
            print(f"Bỏ qua {filename}: Thiếu một hoặc nhiều cột cần thiết.")

In [46]:
import os
import numpy as np
import pandas as pd

def calibrate_sr(sr_values, percentile=5):
    sr_array = np.array(sr_values)
    # Tính giá trị ngưỡng tương ứng với phần trăm đã cho
    threshold = np.percentile(sr_array, percentile)
    
    # Lấy các giá trị thuộc bottom percentile
    bottom_values = sr_array[sr_array <= threshold]
    
    # Tính baseline là trung bình của các giá trị trong bottom percentile
    baseline = np.mean(bottom_values)
    
    # Hiệu chuẩn SR: trừ baseline khỏi tất cả các giá trị SR
    calibrated_sr = sr_array - baseline
    
    return calibrated_sr, baseline

# Đường dẫn tới folder chứa các file CSV
folder_path = r"D:\HThiem\cygnss_to_csv\cygnss_to_csv_0"  # Thay đổi đường dẫn này cho phù hợp

# Lấy danh sách các file CSV trong folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Duyệt qua từng file CSV
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # Đọc file CSV
    df = pd.read_csv(file_path)
    
    # Tính baseline và hiệu chuẩn cho cột SR
    sr_values = df['SR'].values
    calibrated_sr, baseline = calibrate_sr(sr_values)
    
    print(f"Baseline của file {file_name}: {baseline:.4f} dB")
    
    # Thêm cột SR hiệu chuẩn vào DataFrame
    df.loc[:, 'SR_0'] = calibrated_sr
    
    # Lưu file CSV đã hiệu chuẩn, thêm hậu tố '_calibrated' vào tên file
    output_file = os.path.join(folder_path, file_name.replace(".csv", "_calibrated.csv"))
    df.to_csv(output_file, index=False)
    print(f"Đã lưu file hiệu chuẩn: {output_file}\n")


Baseline của file 20240911.csv: 149.1671 dB
Đã lưu file hiệu chuẩn: D:\HThiem\cygnss_to_csv\cygnss_to_csv_0\20240911_calibrated.csv

Baseline của file 20240912.csv: 149.3550 dB
Đã lưu file hiệu chuẩn: D:\HThiem\cygnss_to_csv\cygnss_to_csv_0\20240912_calibrated.csv



In [21]:
import os
import pandas as pd
from glob import glob
check_num = 0
csv_folder = "D:\\output_csv_test\\"

csv_files = glob(os.path.join(csv_folder, "*.csv"))

for file in csv_files:
    df = pd.read_csv(file)

    # Lọc dữ liệu theo điều kiện
    df_filtered = df[
        (df["sp_lat"] > 8.0) &
        (df["sp_lat"] < 24.0) &
        (df["sp_lon"] > 102.0) &
        (df["sp_lon"] < 110.0) &
        (df["sp_inc_angle"] < 65.0) &
        (df["ddm_snr"] > 1.5) &
        (df["sp_rx_gain"] > 0.0)
    ]

    filter_file = file.replace(".csv", "_filtered.csv")
    df_filtered.to_csv(filter_file, index=False)
    check_num += 1
    print('---\n', check_num, filter_file)


---
 1 D:\output_csv_test\20240102_merged_filtered.csv


In [48]:
import os
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

check_num = 0

shp_path = r"D:\New folder\shp\vnm_admbnda_adm0_gov_20200103.shp"

input_csv_folder = r"D:\HThiem\cygnss_to_csv\cygnss_to_csv_0"

output_folder = r"D:\HThiem\cygnss_to_csv\cygnss_to_csv_0"
os.makedirs(output_folder, exist_ok=True)

vn_gdf = gpd.read_file(shp_path)

csv_files = glob.glob(os.path.join(input_csv_folder, "*.csv"))

lat_col = "sp_lat"
lon_col = "sp_lon"

for csv_path in csv_files:
    df = pd.read_csv(csv_path)

    gdf_points = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs=vn_gdf.crs
    )
    
    gdf_clip = gpd.clip(gdf_points, vn_gdf)

    gdf_clip = gdf_clip.copy()

    gdf_clip["lon_clip"] = gdf_clip.geometry.x
    gdf_clip["lat_clip"] = gdf_clip.geometry.y

    gdf_clip.drop(columns=["lon_clip"], inplace=True)
    gdf_clip.drop(columns=["lat_clip"], inplace=True)
    gdf_clip.drop(columns=["geometry"], inplace=True)

    base_name = os.path.basename(csv_path).replace(".csv", "_shp.csv")
    output_csv = os.path.join(output_folder, base_name)

    # Ghi ra file CSV
    gdf_clip.to_csv(output_csv, index=False)
    check_num += 1
    print('---\n', check_num, output_csv)


---
 1 D:\HThiem\cygnss_to_csv\cygnss_to_csv_0\20240911_shp.csv
---
 2 D:\HThiem\cygnss_to_csv\cygnss_to_csv_0\20240911_calibrated_shp.csv
---
 3 D:\HThiem\cygnss_to_csv\cygnss_to_csv_0\20240912_shp.csv
---
 4 D:\HThiem\cygnss_to_csv\cygnss_to_csv_0\20240912_calibrated_shp.csv


In [86]:
import os
import pandas as pd

# Define bit masks for the conditions
S_BAND_POWERED_UP_MASK = 0x00000002
large_sc_attitude_err_MASK = 0x00000008
BLACKBODY_DDM_MASK = 0x00000010
DDM_IS_TEST_PATTERN_MASK = 0x00000080
incorrect_ddmi_antenna_selection_maskk = 0x00000001
noise_floor_cal_error_mask = 0x00000004

def filter_data(df):
    """
    Filters a DataFrame based on the quality_flags column.
    Returns rows where any of the specified bit flags is set.
    """
    filtered = df[
        (df['quality_flags'] & S_BAND_POWERED_UP_MASK == 0) |
        (df['quality_flags'] & SMALL_SC_ATTITUDE_ERR_MASK != 0) |
        (df['quality_flags'] & LARGE_SC_ATTITUDE_ERR_MASK != 0) |
        (df['quality_flags'] & DDM_IS_TEST_PATTERN_MASK != 0)|
        (df['quality_flags_2'] & incorrect_ddmi_antenna_selection_maskk != 0)|
        (df['quality_flags_2'] & noise_floor_cal_error_mask != 0)
    ]
    return filtered

def process_csv_files_in_folder(folder_path, output_folder):
    """
    Processes all CSV files in the given folder. For each CSV file,
    filters the data based on quality_flags and writes the result to
    a new CSV file with a modified name (originalname_filtered.csv) in the output folder.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_csv(file_path)
                # Ensure that quality_flags column is an integer type
                df['quality_flags'] = df['quality_flags'].astype(int)
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                continue

            filtered_df = filter_data(df)
            output_filename = os.path.splitext(filename)[0] + '_filtered.csv'
            output_path = os.path.join(output_folder, output_filename)
            try:
                filtered_df.to_csv(output_path, index=False)
                print(f"Processed {filename} and saved filtered data to {output_path}")
            except Exception as e:
                print(f"Error writing {output_filename}: {e}")

if __name__ == "__main__":
    # Replace these paths with the actual folder paths on your system
    input_folder_path = r'D:\csv_vn_0'
    output_folder_path = r'D:\out_csv_vn'
    
    process_csv_files_in_folder(input_folder_path, output_folder_path)

Processed 20240831_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240831_calibrated_shp_filtered.csv
Processed 20240901_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240901_calibrated_shp_filtered.csv
Processed 20240902_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240902_calibrated_shp_filtered.csv
Processed 20240903_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240903_calibrated_shp_filtered.csv
Processed 20240904_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240904_calibrated_shp_filtered.csv
Processed 20240905_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240905_calibrated_shp_filtered.csv
Processed 20240906_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240906_calibrated_shp_filtered.csv
Processed 20240907_calibrated_shp.csv and saved filtered data to D:\out_csv_vn\20240907_calibrated_shp_filtered.csv
Processed 20240908_calibrated_shp.csv and saved filtered data to D:\out_

In [6]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from rasterio.transform import from_origin
import rasterio
from rasterio.features import rasterize
from scipy.interpolate import Rbf

def load_csv_data(csv_folder):
    """
    Đọc tất cả các file CSV trong thư mục.
    Giả định file CSV có các cột: sp_lat, sp_lon, SR_0
    """
    csv_files = glob.glob(os.path.join(csv_folder, '*.csv'))
    # Đọc các file CSV và trả về list các DataFrame
    dataframes = [pd.read_csv(csv_file) for csv_file in csv_files]
    return dataframes, csv_files

def create_points_gdf(df, crs):
    """
    Tạo GeoDataFrame từ DataFrame với các cột sp_lon và sp_lat
    """
    gdf = gpd.GeoDataFrame(
        df, 
        geometry=gpd.points_from_xy(df.sp_lon, df.sp_lat),
        crs="EPSG:4326"
    )
    return gdf.to_crs(crs)

def rbf_interpolation(points, values, grid_points, epsilon=1):
    """
    Thực hiện nội suy RBF
    points: các điểm quan sát (numpy array với shape (n, 2))
    values: giá trị SR_0 tương ứng với các điểm quan sát (numpy array với shape (n,))
    grid_points: các điểm cần nội suy (numpy array với shape (m, 2))
    epsilon: tham số của hàm RBF
    """
    rbf = Rbf(points[:, 0], points[:, 1], values, function='multiquadric', epsilon=epsilon)
    interpolated_values = rbf(grid_points[:, 0], grid_points[:, 1])
    return interpolated_values

def interpolate_grid(grid, points_gdf, epsilon=1):
    """
    Nội suy cho các ô lưới không có dữ liệu
    """
    # Lấy các điểm và giá trị quan sát
    observed_points = np.array([(point.x, point.y) for point in points_gdf.geometry])
    observed_values = points_gdf['SR_0'].values
    
    # Lấy các điểm centroid của ô lưới
    grid_points = np.array([(point.x, point.y) for point in grid.geometry.centroid])
    
    # Nội suy RBF
    interpolated_values = rbf_interpolation(observed_points, observed_values, grid_points, epsilon)
    
    # Gán giá trị nội suy vào grid
    grid['SR_0'] = interpolated_values
    
    return grid

def export_outputs(grid, output_folder, output_prefix, crs):
    """
    Xuất kết quả ra file CSV và GeoTIFF
    """
    # Tạo thư mục output nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Chuyển đổi hệ tọa độ của centroid về EPSG:4326
    grid_centroids = grid.geometry.centroid.to_crs(epsg=4326)
    
    # Xuất CSV với tọa độ centroid (lat/long) và giá trị SR_0
    output_csv = os.path.join(output_folder, f'{output_prefix}_SR_0_interpolated.csv')
    csv_data = pd.DataFrame({
        'x': grid_centroids.x,
        'y': grid_centroids.y,
        'SR_0': grid['SR_0']
    })
    csv_data.to_csv(output_csv, index=False)
    print(f"Đã xuất file CSV: {output_csv}")
    
    # Xuất GeoTIFF
    output_tif = os.path.join(output_folder, f'{output_prefix}_SR_0_interpolated.tif')
    
    # Tính các thông số cho raster
    bounds = grid.total_bounds
    res = 3000  # độ phân giải 3km
    width = int((bounds[2] - bounds[0]) / res)
    height = int((bounds[3] - bounds[1]) / res)
    transform = from_origin(bounds[0], bounds[3], res, res)
    
    # Chuyển từ vector sang raster
    shapes = ((geom, value) for geom, value in zip(grid.geometry, grid['SR_0']))
    raster = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=np.nan,
        dtype='float32'
    )
    
    # Ghi file GeoTIFF
    with rasterio.open(
        output_tif,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=raster.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        dst.write(raster, 1)
    print(f"Đã xuất file GeoTIFF: {output_tif}")

def main():
    # Đường dẫn input/output
    csv_folder = r"D:\output"
    grid_shp = r"D:\out_csv_vn\Grid_3x3.shp"
    output_folder = r"D:\output\noisuy1"
    
    # Đọc dữ liệu CSV
    print("Đang đọc dữ liệu CSV...")
    dataframes, csv_files = load_csv_data(csv_folder)
    
    # Đọc shapefile lưới
    print("Đang đọc shapefile lưới...")
    grid = gpd.read_file(grid_shp)
    
    # Chuyển đổi hệ tọa độ sang EPSG:32648 (hoặc hệ tọa độ phù hợp khác)
    print("Đang chuyển đổi hệ tọa độ...")
    grid = grid.to_crs("EPSG:32648")
    
    # Loop qua từng DataFrame (tương ứng với từng file CSV)
    for df, csv_file in zip(dataframes, csv_files):
        # Tạo GeoDataFrame từ dữ liệu CSV
        points_gdf = create_points_gdf(df, grid.crs)
        
        # Nội suy cho các ô lưới
        print(f"Đang thực hiện nội suy cho file {csv_file}...")
        grid_interpolated = interpolate_grid(grid.copy(), points_gdf)
        
        # Xuất kết quả
        output_prefix = os.path.splitext(os.path.basename(csv_file))[0]
        print(f"Đang xuất kết quả cho file {csv_file}...")
        export_outputs(grid_interpolated, output_folder, output_prefix, grid.crs)
    
    print("Hoàn thành!")

if __name__ == "__main__":
    main()

Đang đọc dữ liệu CSV...
Đang đọc shapefile lưới...
Đang chuyển đổi hệ tọa độ...
Đang thực hiện nội suy cho file D:\output\20240831_calibrated_shp_filtered.csv...
Đang xuất kết quả cho file D:\output\20240831_calibrated_shp_filtered.csv...
Đã xuất file CSV: D:\output\noisuy1\20240831_calibrated_shp_filtered_SR_0_interpolated.csv
Đã xuất file GeoTIFF: D:\output\noisuy1\20240831_calibrated_shp_filtered_SR_0_interpolated.tif
Đang thực hiện nội suy cho file D:\output\20240901_calibrated_shp_filtered.csv...
Đang xuất kết quả cho file D:\output\20240901_calibrated_shp_filtered.csv...
Đã xuất file CSV: D:\output\noisuy1\20240901_calibrated_shp_filtered_SR_0_interpolated.csv
Đã xuất file GeoTIFF: D:\output\noisuy1\20240901_calibrated_shp_filtered_SR_0_interpolated.tif
Đang thực hiện nội suy cho file D:\output\20240902_calibrated_shp_filtered.csv...
Đang xuất kết quả cho file D:\output\20240902_calibrated_shp_filtered.csv...
Đã xuất file CSV: D:\output\noisuy1\20240902_calibrated_shp_filtered_SR

In [1]:
!pip install scipy

In [8]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from rasterio.transform import from_origin
import rasterio
from rasterio.features import rasterize
from scipy.interpolate import Rbf
from datetime import datetime

def load_csv_data(csv_folder):
    """
    Đọc tất cả các file CSV trong thư mục và trích xuất ngày từ tên file.
    Giả định file CSV có các cột: sp_lat, sp_lon, SR_0
    """
    csv_files = glob.glob(os.path.join(csv_folder, '*.csv'))
    dataframes = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        # Trích xuất ngày từ tên file
        date_str = os.path.basename(csv_file).split('_')[0]
        df['date'] = datetime.strptime(date_str, '%Y%m%d')
        dataframes.append(df)
    return dataframes

def merge_dataframes(dataframes):
    """
    Gộp các DataFrame lại với nhau và nhóm dữ liệu theo ngày
    """
    df = pd.concat(dataframes)
    df = df.sort_values(by='date')
    
    # Gộp dữ liệu cứ 3 ngày liên tiếp thành 1 nhóm
    df['group'] = (df['date'].dt.floor('d').sub(df['date'].dt.floor('d').min()).dt.days // 3).astype(int)
    grouped_dfs = [group for _, group in df.groupby('group')]
    
    return grouped_dfs

def create_points_gdf(df, crs):
    """
    Tạo GeoDataFrame từ DataFrame với các cột sp_lon và sp_lat
    """
    gdf = gpd.GeoDataFrame(
        df, 
        geometry=gpd.points_from_xy(df.sp_lon, df.sp_lat),
        crs="EPSG:4326"
    )
    return gdf.to_crs(crs)

def rbf_interpolation(points, values, grid_points, epsilon=1):
    """
    Thực hiện nội suy RBF
    points: các điểm quan sát (numpy array với shape (n, 2))
    values: giá trị SR_0 tương ứng với các điểm quan sát (numpy array với shape (n,))
    grid_points: các điểm cần nội suy (numpy array với shape (m, 2))
    epsilon: tham số của hàm RBF
    """
    rbf = Rbf(points[:, 0], points[:, 1], values, function='multiquadric', epsilon=epsilon)
    interpolated_values = rbf(grid_points[:, 0], grid_points[:, 1])
    return interpolated_values

def interpolate_grid(grid, points_gdf, epsilon=1):
    """
    Nội suy cho các ô lưới không có dữ liệu
    """
    # Lấy các điểm và giá trị quan sát
    observed_points = np.array([(point.x, point.y) for point in points_gdf.geometry])
    observed_values = points_gdf['SR_0'].values
    
    # Lấy các điểm centroid của ô lưới
    grid_points = np.array([(point.x, point.y) for point in grid.geometry.centroid])
    
    # Nội suy RBF
    interpolated_values = rbf_interpolation(observed_points, observed_values, grid_points, epsilon)
    
    # Gán giá trị nội suy vào grid
    grid['SR_0'] = interpolated_values
    
    return grid

def export_outputs(grid, output_folder, output_prefix, crs):
    """
    Xuất kết quả ra file CSV và GeoTIFF
    """
    # Tạo thư mục output nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Chuyển đổi tọa độ centroid sang hệ tọa độ WGS84
    grid_centroids = grid.geometry.centroid.to_crs(epsg=4326)
    
    # Xuất CSV với tọa độ lat, lon và giá trị SR_0
    output_csv = os.path.join(output_folder, f'{output_prefix}_SR_0_interpolated.csv')
    csv_data = pd.DataFrame({
        'lat': grid_centroids.y,
        'lon': grid_centroids.x,
        'SR_0': grid['SR_0']
    })
    csv_data.to_csv(output_csv, index=False)
    print(f"Đã xuất file CSV: {output_csv}")
    
    # Xuất GeoTIFF
    output_tif = os.path.join(output_folder, f'{output_prefix}_SR_0_interpolated.tif')
    
    # Tính các thông số cho raster
    bounds = grid.total_bounds
    res = 3000  # độ phân giải 3km
    width = int((bounds[2] - bounds[0]) / res)
    height = int((bounds[3] - bounds[1]) / res)
    transform = from_origin(bounds[0], bounds[3], res, res)
    
    # Chuyển từ vector sang raster
    shapes = ((geom, value) for geom, value in zip(grid.geometry, grid['SR_0']))
    raster = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=np.nan,
        dtype='float32'
    )
    
    # Ghi file GeoTIFF
    with rasterio.open(
        output_tif,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=raster.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        dst.write(raster, 1)
    print(f"Đã xuất file GeoTIFF: {output_tif}")

def main():
    # Đường dẫn input/output
    csv_folder = r"D:\output"
    grid_shp = r"D:\out_csv_vn\Grid_3x3.shp"
    output_folder = r"D:\output\noisuy"
    
    # Đọc dữ liệu CSV
    print("Đang đọc dữ liệu CSV...")
    dataframes = load_csv_data(csv_folder)
    
    # Gộp dữ liệu cứ 3 ngày liên tiếp thành 1 nhóm
    print("Đang gộp dữ liệu...")
    grouped_dataframes = merge_dataframes(dataframes)
    
    # Đọc shapefile lưới
    print("Đang đọc shapefile lưới...")
    grid = gpd.read_file(grid_shp)
    
    # Chuyển đổi hệ tọa độ sang EPSG:32648 (hoặc hệ tọa độ phù hợp khác)
    print("Đang chuyển đổi hệ tọa độ...")
    grid = grid.to_crs("EPSG:32648")
    
    # Loop qua từng nhóm DataFrame (kết quả của việc gộp dữ liệu)
    for i, df in enumerate(grouped_dataframes):
        # Tạo GeoDataFrame từ dữ liệu CSV
        points_gdf = create_points_gdf(df, grid.crs)
        
        # Nội suy cho các ô lưới
        print(f"Đang thực hiện nội suy cho nhóm dữ liệu {i+1}...")
        grid_interpolated = interpolate_grid(grid.copy(), points_gdf)
        
        # Xuất kết quả
        output_prefix = f'group_{i+1}'
        print(f"Đang xuất kết quả cho nhóm dữ liệu {i+1}...")
        export_outputs(grid_interpolated, output_folder, output_prefix, grid.crs)
    
    print("Hoàn thành!")

if __name__ == "__main__":
    main()

Đang đọc dữ liệu CSV...
Đang gộp dữ liệu...
Đang đọc shapefile lưới...
Đang chuyển đổi hệ tọa độ...
Đang thực hiện nội suy cho nhóm dữ liệu 1...
Đang xuất kết quả cho nhóm dữ liệu 1...
Đã xuất file CSV: D:\output\noisuy\group_1_SR_0_interpolated.csv
Đã xuất file GeoTIFF: D:\output\noisuy\group_1_SR_0_interpolated.tif
Đang thực hiện nội suy cho nhóm dữ liệu 2...
Đang xuất kết quả cho nhóm dữ liệu 2...
Đã xuất file CSV: D:\output\noisuy\group_2_SR_0_interpolated.csv
Đã xuất file GeoTIFF: D:\output\noisuy\group_2_SR_0_interpolated.tif
Đang thực hiện nội suy cho nhóm dữ liệu 3...
Đang xuất kết quả cho nhóm dữ liệu 3...
Đã xuất file CSV: D:\output\noisuy\group_3_SR_0_interpolated.csv
Đã xuất file GeoTIFF: D:\output\noisuy\group_3_SR_0_interpolated.tif
Đang thực hiện nội suy cho nhóm dữ liệu 4...
Đang xuất kết quả cho nhóm dữ liệu 4...
Đã xuất file CSV: D:\output\noisuy\group_4_SR_0_interpolated.csv
Đã xuất file GeoTIFF: D:\output\noisuy\group_4_SR_0_interpolated.tif
Hoàn thành!
